In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader

In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../data/dataset/',
                                 transform=T.ToSparseTensor())

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [5]:
data = dataset[0]

In [6]:
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [7]:
data = data.to(device)

In [8]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [9]:
evaluator = Evaluator(name='ogbn-arxiv')

In [10]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs.log_softmax(dim=-1)

In [11]:
data['x'].shape[1]

128

In [12]:
data['y'][:, 0].unique().shape[0]

40

In [13]:
n_iters = 3000
epochs = 10000
log_steps = 100
input_dim = data['x'].shape[1]
output_dim = data['y'][:, 0].unique().shape[0]
lr_rate = 0.001

In [14]:
model = LogisticRegression(input_dim, output_dim).to(device)

In [15]:
criterion = torch.nn.CrossEntropyLoss()

In [16]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [17]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 2.8646, Train: 23.81%, Valid: 20.38% Test: 17.51%
Epoch: 200, Loss: 2.6096, Train: 28.07%, Valid: 27.38% Test: 24.73%
Epoch: 300, Loss: 2.4279, Train: 34.03%, Valid: 35.03% Test: 32.51%
Epoch: 400, Loss: 2.2880, Train: 38.70%, Valid: 40.17% Test: 37.69%
Epoch: 500, Loss: 2.1785, Train: 42.06%, Valid: 43.60% Test: 41.29%
Epoch: 600, Loss: 2.0917, Train: 44.41%, Valid: 45.84% Test: 43.44%
Epoch: 700, Loss: 2.0219, Train: 46.11%, Valid: 47.12% Test: 45.15%
Epoch: 800, Loss: 1.9650, Train: 47.39%, Valid: 48.23% Test: 46.26%
Epoch: 900, Loss: 1.9180, Train: 48.27%, Valid: 49.14% Test: 47.09%
Epoch: 1000, Loss: 1.8788, Train: 49.05%, Valid: 49.78% Test: 47.70%
Epoch: 1100, Loss: 1.8456, Train: 49.64%, Valid: 50.34% Test: 48.19%
Epoch: 1200, Loss: 1.8173, Train: 50.12%, Valid: 50.80% Test: 48.61%
Epoch: 1300, Loss: 1.7929, Train: 50.55%, Valid: 51.14% Test: 49.01%
Epoch: 1400, Loss: 1.7717, Train: 50.91%, Valid: 51.48% Test: 49.30%
Epoch: 1500, Loss: 1.7532, Train: 51.26%, V